# Importando as bibliotecas

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import pymysql

# Conexão com o Banco de dados

In [2]:
def mysql_connection(host, user, password, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
    return engine.connect()

In [3]:
connection =  mysql_connection("localhost", "root", "teste2236","analise_risco" )

# Criando os DataFrames

## Dados Mutuários

In [18]:
dados_mutuarios = pd.read_sql_query("SELECT * FROM dados_mutuarios", connection)
dados_mutuarios.columns = ["id_pessoal","idade","salario","situacao_propriedade","tempo_trabalho"]
dados_mutuarios.head()

,id_pessoal,idade,salario,situacao_propriedade,tempo_trabalho
0,tad8vkTn-vp4l,33.0,48000.0,Rent,17.0
1,dEILQqp2-lgLx,24.0,75000.0,Own,0.0
2,Az11sWWw-SdFA,29.0,28800.0,Rent,5.0
3,YDovEwBH-0eek,23.0,75000.0,Mortgage,7.0
4,lNsbfwlP-gzOE,26.0,35000.0,Rent,0.0


In [25]:
print(f'O DataFrame com os dados mutuários tem {dados_mutuarios.shape[0]} linhas e {dados_mutuarios.shape[1]} colunas')

O DataFrame com os dados mutuários tem 34489 linhas e 5 colunas


In [24]:
dados_mutuarios['id_pessoal'].value_counts()

                 4
tad8vkTn-vp4l    1
k6qkYxHh-eGq9    1
rFwCWz7t-axj5    1
9rpWAdiI-UMFP    1
                ..
L5miIhEg-JqSp    1
q6HGSybf-oXL1    1
18GQojFZ-t8c4    1
6qokl7TM-pt7S    1
IoRwjwOl-lQ9u    1
Name: id_pessoal, Length: 34486, dtype: int64

Ao analisar a coluna id_pessoal, foram identificadas 4 entradas vazias. Isso pode resultar em perda de informações importantes ao cruzar os dados destes 4 clientes, dificultando a montagem de um perfil preciso para os modelos de Machine Learning. Será necessário avaliar essas entradas vazias posteriormente e tomar uma decisão quanto a sua inclusão ou exclusão.

In [26]:
dados_mutuarios['idade'].value_counts()

23.0     4079
22.0     3809
24.0     3731
25.0     3175
26.0     2610
27.0     2242
28.0     1932
29.0     1761
30.0     1385
21.0     1294
31.0     1205
32.0     1013
33.0      910
34.0      738
35.0      642
36.0      573
37.0      505
38.0      388
39.0      314
40.0      284
41.0      252
42.0      198
43.0      169
44.0      148
45.0      113
46.0      100
47.0       95
48.0       76
50.0       58
49.0       50
51.0       40
52.0       37
53.0       34
54.0       27
58.0       20
55.0       20
57.0       15
60.0       15
56.0       15
20.0       15
64.0       10
66.0       10
65.0        9
61.0        9
70.0        8
62.0        7
59.0        5
69.0        5
144.0       3
73.0        3
63.0        3
78.0        2
123.0       2
76.0        1
94.0        1
67.0        1
80.0        1
84.0        1
Name: idade, dtype: int64

In [30]:
dados_mutuarios.query('idade > 100 or idade < 18')

,id_pessoal,idade,salario,situacao_propriedade,tempo_trabalho
11812,9KZ6344x-RDHc,123.0,80004.0,Rent,2.0
16030,5fi1GWSO-oQcb,144.0,200000.0,Mortgage,4.0
16051,PBb5WjDd-yZ2g,123.0,78000.0,Rent,7.0
26573,TwZcn29T-CPBK,144.0,250000.0,Rent,4.0
28756,ZpG5kPRS-oxN1,144.0,6000000.0,Mortgage,12.0


Existem algumas entradas inconsistentes na variável idade. Existem alguns usuários cadastrados no banco de dados, com mais de cem anos.

In [31]:
dados_mutuarios['salario'].value_counts()

60000.0    1097
30000.0     886
50000.0     810
40000.0     694
45000.0     618
           ... 
41725.0       1
58570.0       1
74250.0       1
36225.0       1
51888.0       1
Name: salario, Length: 4295, dtype: int64

In [32]:
dados_mutuarios['situacao_propriedade'].value_counts()

Rent        17237
Mortgage    14094
Own          2717
              331
Other         110
Name: situacao_propriedade, dtype: int64

Existem 331 entradas sem informação sobre a situação da propriedade.

In [33]:
dados_mutuarios['tempo_trabalho'].value_counts()

0.0      4317
2.0      4044
3.0      3615
5.0      3091
1.0      3051
4.0      3006
6.0      2799
7.0      2312
8.0      1774
9.0      1434
11.0      790
10.0      721
12.0      596
13.0      441
14.0      351
15.0      246
16.0      171
17.0      137
18.0      110
19.0       68
20.0       44
21.0       39
22.0       19
24.0       11
23.0       11
25.0        9
26.0        7
27.0        5
28.0        4
31.0        4
123.0       2
30.0        2
41.0        1
38.0        1
29.0        1
34.0        1
Name: tempo_trabalho, dtype: int64

## Emprestimos

In [19]:
emprestimos = pd.read_sql_query("SELECT * FROM emprestimos", connection)
emprestimos.columns = ["id_emprestimo","motivo","pontuacao","valor","taxa_juros","inadimplencia","renda_percentual"]
emprestimos.head()

,id_emprestimo,motivo,pontuacao,valor,taxa_juros,inadimplencia,renda_percentual
0,NMgFWFWN-2yGG,Homeimprovement,C,20000.0,13.22,0.0,0.22
1,08Ts3XSa-KkvE,Venture,C,6000.0,13.49,0.0,0.04
2,xo79pAME-elmp,Personal,A,8000.0,6.17,0.0,0.13
3,IB6lBS2h-nWhz,Homeimprovement,B,9600.0,10.08,0.0,0.16
4,CONLtJHn-1yde,Medical,A,6500.0,8.49,0.0,0.30


In [34]:
emprestimos.id_emprestimo.value_counts()

NMgFWFWN-2yGG    1
y8MsBbLa-aSFe    1
pDhzNJ3y-UMgu    1
wj6WcdT5-7HBi    1
OG3lUkqF-RYqF    1
                ..
yw9L2T32-rwGD    1
3gislqGQ-BAUW    1
LwdHLn2b-jJ4h    1
X1iMYKFZ-Y9vt    1
GhqTLrER-Zhig    1
Name: id_emprestimo, Length: 34489, dtype: int64

In [35]:
emprestimos.motivo.value_counts()

Education            6771
Medical              6380
Venture              5977
Personal             5782
Debtconsolidation    5476
Homeimprovement      3791
                      312
Name: motivo, dtype: int64

In [36]:
emprestimos.pontuacao.value_counts()

A    11324
B    10955
C     6765
D     3810
E     1007
       310
F      254
G       64
Name: pontuacao, dtype: int64

In [38]:
emprestimos.valor.value_counts()

10000.0    2798
5000.0     2122
12000.0    1905
6000.0     1899
15000.0    1582
           ... 
5725.0        1
1225.0        1
7225.0        1
750.0         1
30600.0       1
Name: valor, Length: 753, dtype: int64

In [39]:
emprestimos.taxa_juros.value_counts()

10.99    795
7.51     789
7.88     680
7.49     679
5.42     622
        ... 
20.52      1
17.44      1
20.48      1
17.34      1
16.71      1
Name: taxa_juros, Length: 348, dtype: int64

In [40]:
emprestimos.columns

Index(['id_emprestimo', 'motivo', 'pontuacao', 'valor', 'taxa_juros',
       'inadimplencia', 'renda_percentual'],
      dtype='object')

In [41]:
emprestimos.inadimplencia.value_counts()

0.0    26696
1.0     7450
Name: inadimplencia, dtype: int64

In [43]:
emprestimos.renda_percentual.value_counts()

0.10    1615
0.13    1550
0.08    1504
0.11    1459
0.07    1456
        ... 
0.66       2
0.83       1
0.72       1
0.62       1
0.78       1
Name: renda_percentual, Length: 77, dtype: int64

## Histórico 

In [20]:
historico_banco = pd.read_sql_query("SELECT * FROM historicos_banco", connection)
historico_banco.columns = ["id_historico","inadimplente","tempo_credito"]
historico_banco.head()

,id_historico,inadimplente,tempo_credito
0,ZNWGR0Ap-FDt2,N,9.0
1,wqALDH4x-mnzE,N,7.0
2,57O09sJw-NBKj,N,6.0
3,pH50QiTJ-0jqz,Y,10.0
4,3voX5CIK-tCxE,N,12.0


In [44]:
historico_banco.inadimplente.value_counts()

N    28122
Y     6000
       367
Name: inadimplente, dtype: int64

In [45]:
historico_banco.tempo_credito.value_counts()

2.0     6312
3.0     6290
4.0     6280
7.0     2008
8.0     2003
5.0     2000
9.0     1992
10.0    1973
6.0     1951
14.0     525
12.0     510
11.0     488
16.0     476
13.0     475
15.0     470
17.0     431
20.0      34
24.0      32
28.0      30
27.0      25
30.0      24
23.0      24
22.0      23
21.0      22
19.0      21
18.0      20
25.0      19
26.0      16
29.0      14
Name: tempo_credito, dtype: int64

## Ids

In [11]:
ids = pd.read_sql_query("SELECT * FROM ids", connection)
ids.columns = ["id_pessoa","id_emprestimo","id_historico"]
ids.head()

,id_pessoa,id_emprestimo,id_historico
0,JAz2tRvQ-14cy,pjEu6snv-4t33,FIgNSkrG-wen7
1,MZSefq9m-loYS,Mfn0Csx4-LGM3,QPDJUQfQ-DJqK
2,cRItaaPB-pRvT,Ks4nz74P-z6Iq,hzAXqSDh-ay7Q
3,5jTDCfXS-zaAq,ZFlwW08i-I3Al,LiAivrS4-J5DE
4,Hbz9PtI3-tOFs,bFMFchcv-UwQW,PkBixMJ3-3bad


In [51]:
for coluna in ids.columns:
    print(f'Da variável: {coluna}')
    print(ids[coluna].value_counts())
    print()

Da variável: id_pessoa
                 4
JAz2tRvQ-14cy    1
Hm07DgPA-XkMt    1
XzKyorQH-uSCc    1
ftJIyo1P-uHL7    1
                ..
v5bWKyj1-qyBN    1
AvpUlHD0-CZcw    1
JlUBw7JQ-oncA    1
o6JMgTsH-4ve6    1
2OjPj0Cv-Ohe3    1
Name: id_pessoa, Length: 34486, dtype: int64

Da variável: id_emprestimo
pjEu6snv-4t33    1
CaTUCyAH-Oy5e    1
dmC5eJcg-TjRY    1
I4hOtl6q-CHeq    1
sL5yMaoq-u9hs    1
                ..
Caj8UitA-y0kX    1
MUDgAINY-Ph3w    1
JCyN26Ta-HSKv    1
eyvZB1ho-FxFG    1
XCwnpXOc-QxGQ    1
Name: id_emprestimo, Length: 34489, dtype: int64

Da variável: id_historico
FIgNSkrG-wen7    1
1RsoyYo1-tPZf    1
9XjPuuic-PnJL    1
DefUkcll-4YTU    1
eAKkgCYJ-9GAv    1
                ..
rzsXZxSR-jvSi    1
ZOvKdko9-eulw    1
dfUvxSdC-4USR    1
iqhydGgE-tPmd    1
lUcqMLJE-pqLe    1
Name: id_historico, Length: 34489, dtype: int64



# Junção dos DataFrames

In [17]:
df_merged = ids.merge(historico_banco, left_on="id_historico", right_on="id_historico")
df_merged.head()

,id_pessoa,id_emprestimo,id_historico,inadimplente,tempo_credito
0,JAz2tRvQ-14cy,pjEu6snv-4t33,FIgNSkrG-wen7,Y,2.0
1,MZSefq9m-loYS,Mfn0Csx4-LGM3,QPDJUQfQ-DJqK,N,4.0
2,cRItaaPB-pRvT,Ks4nz74P-z6Iq,hzAXqSDh-ay7Q,N,6.0
3,5jTDCfXS-zaAq,ZFlwW08i-I3Al,LiAivrS4-J5DE,,11.0
4,Hbz9PtI3-tOFs,bFMFchcv-UwQW,PkBixMJ3-3bad,Y,6.0
